In [ ]:
!pip3 install transformers
!pip3 install pandas
!pip3 install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 16.0 MB/s 
     |████████████████████████████████| 182 kB 67.8 MB/s 
     |████████████████████████████████| 7.6 MB 58.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train.csv')
df["intput"] = df["sentence1"]+df["sentence2"]+df["sentence3"]+df["sentence4"]
df = df.loc[:, df.columns.intersection(['storytitle','intput'])]
df = df.dropna(axis="rows")

df = df.head(5000)
df.columns = df.columns.str.strip()

In [ ]:
class Story(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.input = []

        for row in df['intput']:
          self.input.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))               
        # if truncate:
        #     self.lyrics = self.lyrics[:20000]
        self.input_count = len(self.input)
        
    def __len__(self):
        return self.input_count

    def __getitem__(self, item):
        return self.input[item]

# class Story(Dataset):  
#     def __init__(self, txt_list, truncate=False, gpt2_type="gpt2", max_length=1024):


#         self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>',
#                                           eos_token='<|endoftext|>', pad_token='<|pad|>')
#         self.input = []

#         for row in df['intput']:
#           s = '<|startoftext|>' + row + '<|endoftext|>'
#           self.input.append(torch.tensor(
#                 self.tokenizer.encode(f"<|endoftext|>{row[:max_length]}<|endoftext|>")
#             ))               
#         # if truncate:
#         #     self.lyrics = self.lyrics[:20000]
#         self.input_count = len(self.input)
        
#     def __len__(self):
#         return self.input_count

#     def __getitem__(self, item):
#         return self.input[item]
    
dataset = Story(df['intput'], truncate=True, gpt2_type="gpt2") 
print(len(dataset))

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

5000


In [ ]:
#Get the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=10, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 20
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [ ]:
len(dataset)

5000

In [ ]:
model = train(dataset, model, tokenizer)

Training epoch 0
0


5000it [06:02, 13.81it/s]


Training epoch 1
tensor(0.6149, device='cuda:0', grad_fn=<NllLossBackward0>)


5000it [06:01, 13.84it/s]


Training epoch 2
tensor(0.4451, device='cuda:0', grad_fn=<NllLossBackward0>)


...

5000it [06:01, 13.82it/s]


Training epoch 8
tensor(0.5078, device='cuda:0', grad_fn=<NllLossBackward0>)


5000it [05:59, 13.89it/s]


Training epoch 9
tensor(0.4376, device='cuda:0', grad_fn=<NllLossBackward0>)


5000it [05:53, 14.14it/s]


In [ ]:
torch.save(model, '/content/drive/MyDrive/newGPT2Model6')
